# Learning to use widgets

### Changelog:

* 15 Feb 2016
    * Discovered that `IPython.html.widgets` have been replaced with `ipywidgets`
        * Widget classes, such as `IntSliderWidget`, are now referred to as simply `IntSlider`
    * Updated [initial notebook](http://nbviewer.jupyter.org/github/ipython/ipywidgets/blob/master/examples/Using%20Interact.ipynb) to point to the latest version which also include revised and new material
    * Attempting to install `ipywidgets` triggered a much bigger update of notebook, conda, etc, etc
    * Master site is at https://github.com/ipython/ipywidgets.  This is still very active.  `ipywidgets` has only recently been released as a library and the docs at https://ipywidgets.readthedocs.org are a bit sparse - no API details as yet
* 14 Feb 2016:
    * Experimented with function annotation format
        * Issues with slider parameter initialization
        * Added Changelog and Table of Contents
* 13 Feb 2016:
    * Downloaded [initial notebook](http://nbviewer.jupyter.org/github/quantopian/ipython/blob/master/examples/Interactive%20Widgets/Using%20Interact.ipynb)
    * Deleted Python 2 material as irrelevant
    * Customized and extended examples: `get_minutes()`, etc
    * Re-discovered ouput supression trick: `;` at end of `interact()` call
    


----

### Table of Contents

* [Using Interact](#Using-Interact)
    * [Basic interact](#Basic-interact)
    * [interact decorator](#interact-decorator)
    * [Fixing arguments using fixed](#Fixing-arguments-using-fixed)
    * [Widget abbreviations](#Widget-abbreviations)
    * [Issue: setting default slider values](#Issue:-setting-default-slider-values)
    * [Correction: setting default slider values](#Correction:-setting-default-slider-values )
    * [Note: possible development flow](#Note:-possible-development-flow)
        * [Start with a regular function def and call ...](#Start-with-a-regular-function-def-and-call-...)
        * [Then replace the call with the interact decorator ...](#Then-replace-the-call-with-the-interact-decorator-...)
    * [Note: closing the slider widget window, leaves the output result visible ...](#Note:-closing-the-slider-widget-window,-leaves-the-output-result-visible-...)
    * [More on min, max, step and initial values](#More-on-min,-max,-step-and-initial-values)    
    * [Dropdown Menus](#Dropdown-Menus)
        * [Note: the default does not seem to work dicts](#Note:-the-default-does-not-seem-to-work-dicts)
        * [Note: suppressing unwanted output](#Note:-suppressing-unwanted-output)
    * [Using function annotations with interact](#Using-function-annotations-with-interact)
    * [Trying annotations with the get_minutes example](#Trying-annotations-with-the-get_minutes-example)
    * [Issue: Annotations with initial values](#Issue:-Annotations-with-initial-values)
* [interactive](#interactive)
    * [Using Interactive with get_minutes](#Using-Interactive-with-get_minutes)
        * [Note: We did not need to call display to see the widgets](#Note:-We-did-not-need-to-call-display-to-see- the-widgets)
    * [Note: We added a return statement to get_minutes to enable w_get_mins.result](#Note:-We-added-a-return-statement-to-get_minutes-to-enable-w_get_mins.result)


----

### Installed version of `ipywidgets`

In [1]:
!conda list ipywidgets

/bin/sh: conda: command not found


## Using `interact`

The `interact` function (`IPython.html.widgets.interact`) automatically creates user interface (UI) controls for exploring code and data interactively. It is the easiest way to get started using IPython's widgets.

In [2]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

### Basic `interact`

At the most basic level, `interact` autogenerates UI controls for function arguments, and then calls the function with those arguments when you manipulate the controls interactively. To use `interact`, you need to define a function that you want to explore. Here is a function that prints its only argument `x`.

In [3]:
def sqr(x):
    return(x**2)

When you pass this function as the first argument to `interact` along with an integer keyword argument (`x=10`), a slider is generated and bound to the function.

In [4]:
interact(sqr, x=50);

1681

When you move the slider, the function is called and the current value of `x` is printed.

If you pass `True` or `False`, `interact` will generate a checkbox:

In [6]:
def print_status(married):
    print("Marital status: {}".format(married))

interact(print_status, married=False);

Marital status: False


If you pass a string, `interact` will generate a text area.

In [8]:
def print_name(name):
    print("Your name is: {}".format(name.upper()))


interact(print_name, name='Enter your name here');

Your name is: ENTER YOUR NAME HERE


----

### `interact` decorator

`interact` can also be used as a decorator. This allows you to define a function and interact with it in a single shot. As this example shows, `interact` also works with functions that have multiple arguments.

In [9]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

(True, 1.4)

In [10]:
@interact(weeks=51, days=6, hours=23)
def get_minutes(weeks, days, hours):
    minutes = 60 * (hours + (24 * (days + (7 * weeks))))
    print("{} weeks, {} days plus {} hours equal ... {} minutes". format(weeks, days, hours, minutes ))

51 weeks, 6 days plus 23 hours equal ... 524100 minutes


### Fixing arguments using `fixed`

There are times when you may want to explore a function using `interact`, but fix one or more of its arguments to specific values. This can be accomplished by wrapping values with the `fixed` function.

In [11]:
def h(p, q):
    return(p, q)

When we call `interact`, we pass `fixed(20)` for q to hold it fixed at a value of `20`.

In [12]:
interact(h, p=1, q=fixed(20));

(1, 20)

Notice that a slider is only produced for `p` as the value of `q` is fixed.

### Widget abbreviations

When you pass an integer valued keyword argument (`x=10`) to `interact`, it generates an integer valued slider control with a range of $[-10,+3\times10]$.  
In this case `10` is an *abbreviation* for an actual slider widget:

```python
IntSliderWidget(min=-10,max=30,step=1,value=10)
```

In fact, we can get the same result if we pass this `IntSlider` as the keyword argument for `x`:

In [13]:
def f(x):
    return(x)

interact(f, x=widgets.IntSlider(min=0,max=20,step=5,value=10));

10

This examples clarifies how `interact` proceses its keyword arguments:

1. If the keyword argument is `Widget` instance with a `value` attribute, that widget is used.  
    Any widget with a `value` attribute can be used, even custom ones.

2. Otherwise, the value is treated as a *widget abbreviation* that is converted to a widget before it is used.

The following table gives an overview of different widget abbreviations:

|Keyword argument                                                   | Widget      |
|-------------------------------------------------------------------|-------------|
|`True` or `False`                                                  | Checkbox    |
| 'Hi there'                                                        | Textarea    |
| `value` or `(min,max)` or `(min,max,step)` if integers are passed | IntSlider   |
| `value` or `(min,max)` or `(min,max,step)` if floats are passed   | FloatSlider |
| `('orange','apple')` or `{'one':1,'two':2}`                       | Dropdown    |

In [14]:
@interact(years=widgets.IntSlider(min=0,max=150,step=1,value=0),
          weeks=widgets.IntSlider(min=0,max=51,step=1,value=0),
          days=widgets.IntSlider(min=0,max=6,step=1,value=0),
          hours=widgets.IntSlider(min=0,max=23,step=1,value=1))
def get_minutes(years, weeks, days, hours):
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes ))

0 years, 0 weeks, 0 days plus 17 hours equal ... 1020 minutes


In [15]:
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
def get_minutes(years, weeks, days, hours):
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

75 years, 25 weeks, 3 days plus 11 hours equal ... 5872980 minutes


### Issue: setting default slider values

We cannot add the initial value as a 4th entry in the tuples above for the parameters of the various widgets: `years`, `weeks`, `days` and `hours`

It seems odd that 

``` python
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
```

is ok, but 

``` python
@interact(years=(0,150,1,0), weeks=(0,51,1,0), days=(0,6,1,0), hours=(0,23,1,1))
```

throws the following exception!

``` python
ValueError: (0, 150, 1, 0) cannot be transformed to a Widget

```

In [16]:
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
def get_minutes(years=0, weeks=0, days=0, hours=1):
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

0 years, 0 weeks, 0 days plus 1 hours equal ... 60 minutes


### Correction: setting slider values

So the way to set the initial values of the sliders is to set the defaults in the `get_minutes` function parameters.

```python
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
def get_minutes(years=0, weeks=0, days=0, hours=1):
...
```
This makes sense and is more 'Pythonic'

### Note: one possible development flow

#### Start with a regular function def and call ...

In [17]:
def get_minutes(years=0, weeks=0, days=0, hours=1):
    'Calculates number of minutes in so many years, + weeks, + days'
    
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

get_minutes(0,0,0,5)

0 years, 0 weeks, 0 days plus 5 hours equal ... 300 minutes


#### Then replace the call with the `interact` decorator ...

In [18]:
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
def get_minutes(years=0, weeks=0, days=0, hours=1):
    'Calculates number of minutes in so many years, + weeks, + days'
    
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

#get_minutes(0,0,0,2)

0 years, 0 weeks, 0 days plus 1 hours equal ... 60 minutes


#### Note: `closing` the slider widget window, leaves the output result visible ...

In [19]:
@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))
def get_minutes(years=0, weeks=0, days=0, hours=1):
    'Calculates number of minutes in so many years, + weeks, + days'
    
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

0 years, 0 weeks, 0 days plus 1 hours equal ... 60 minutes


----

### More on `min`, `max`, `step` and `initial values`

You have seen how the checkbox and textarea widgets work above. Here, more details about the different abbreviations for sliders and dropdowns are given.

If a 2-tuple of integers is passed `(min,max)` a integer valued slider is produced with those minimum and maximum (inclusive) values. In this case, the default step size of `1` is used.

In [20]:
interact(f, x=(0,4));

2

If a 3-tuple of integers is passed `(min,max,step)` the step size can also be set.

In [21]:
interact(f, x=(0,8,2));

4

A float valued slider is produced if the elements of the tuples are floats. Here the minimum is `0.0`, the maximum is `10.0` and step size is `0.1` (the default).

In [22]:
interact(f, x=(0.0,10.0));

5.0

The step size can be changed by passing a 3rd element in the tuple.

In [23]:
interact(f, x=(0.0,10.0,0.01));

4.99

For both integer and float valued sliders, you can pick the initial value of the widget by passing a default keyword argument to the underlying Python function. Here we set the initial value of a float slider to `5.5`.

In [24]:
@interact(x=(0.0,20.0,0.5))
def h(x=5.5):
    return(x)

5.5

### Dropdown Menus

Dropdown menus can be produced by passing a tuple of strings. In this case, the strings are both used as the names in the dropdown menu UI and passed to the underlying Python function.

In [25]:
def f(x='oranges'):
    print(x)

interact(f, x=('apples','oranges'));

oranges


If you want a dropdown menu that passes non-string values to the Python function, you can pass a dictionary. The keys in the dictionary are used for the names in the dropdown menu UI and the values are the arguments that are passed to the underlying Python function.

In [26]:
def days_to_dias(day='Thursday'):
    print(day)

interact(days_to_dias, day={'Monday': 'lunes', 'Tuesday': 'martes', 'Wednesday': 'miércoles',
                             'Thursday': 'jueves', 'Friday': 'viernes', 'Saturday': 'sábado',
                             'Sunday': 'domingo'});

jueves


#### Note: the default does not seem to work dicts

In [27]:
days_in_month = {'January': 31, 'February': 28, 'March': 31, 'April': 30, 'May': 31, 'June': 30,
             'July': 31, 'August': 31, 'September': 30, 'October': 31, 'November': 30, 'December': 31}

def print_days_in_month(month='September'):
    print('There {} days in this month'.format(month))
    
    
interact(print_days_in_month, month=days_in_month);

There 30 days in this month


#### Note: suppressing unwanted output

The `;` at the end of the function call

```python
interact(print_days_in_month, month=days_in_month);
```

It suppresses the output line after the widget area

```python
<function __main__.print_days_in_month>
```

### Using function annotations with `interact`

If you are using Python 3, you can also specify widget abbreviations using [function annotations](https://docs.python.org/3/tutorial/controlflow.html#function-annotations). This is a convenient approach allows the widget abbreviations to be defined with a function.

Define a function with an checkbox widget abbreviation for the argument `x`.

In [28]:
def f(x:True):
    return(x)

Then, because the widget abbreviation has already been defined, you can call `interact` with a single argument.

In [29]:
interact(f);

True

#### Trying annotations with the `get_minutes` example

In [30]:
#@interact(years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))

def get_minutes(years:(0,150,1), weeks:(0,51,1), days:(0,6,1), hours:(0,23,1)):
    'Calculates number of minutes in so many years, + weeks, + days'
    
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))

interact(get_minutes);  # `;` suppresses output of <function __main__.get_minutes>

75 years, 25 weeks, 3 days plus 11 hours equal ... 5872980 minutes


#### Issue: Annotations with initial values

The annotation can be a tuple of 3 values (min, max, step) but the initial vaue cannot be added to the tuple as a 4th element.

Nor can the initial value be applied in the regular way of specifying the default value of a function parameter when using annottaions as shown in the [Python docs](https://docs.python.org/3/tutorial/controlflow.html#function-annotations) ...


```python
>>> def f(ham: str, eggs: str = 'eggs') -> str:
...     print("Annotations:", f.__annotations__)
...     print("Arguments:", ham, eggs)
...     return ham + ' and ' + eggs
```

This version:
```python
def get_minutes(years:(0,150,1)=1, weeks:(0,51,1), days:(0,6,1), hours:(0,23,1)):
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))
```

raises the following error ...

```python
  File "<ipython-input-208-e77735168217>", line 3
    def get_minutes(years:(0,150,1)=1, weeks:(0,51,1), days:(0,6,1), hours:(0,23,1)):
    
SyntaxError: non-default argument follows default argument
```

----

## `interactive()`

In addition to `interact()` IPython provides another function, `interactive()`.

`interactive()` is useful when

* you want to reuse the widgets that are produced

* you want to **access the data that is bound to the UI controls**

Here is a function that returns the sum of its two arguments.

In [31]:
def sum(a, b):
    return a+b

Unlike `interact`, `interactive` returns a `Widget` instance rather than immediately displaying the widget.

In [32]:
w_sum = interactive(sum, a=10, b=20)

The widget is a `Box`, which is a container for other widgets.

In [33]:
type(w_sum)

ipywidgets.widgets.widget_box.Box

The children of the `Box` are two integer valued sliders produced by the widget abbreviations above.

In [34]:
w_sum.children

(<ipywidgets.widgets.widget_int.IntSlider at 0x317e8a50>,
 <ipywidgets.widgets.widget_int.IntSlider at 0x317e89d0>)

To actually display the widgets, you can use IPython's `display` function.

In [35]:
from IPython.display import display
display(w_sum)

30

At this point, the UI controls work just like they would if `interact` had been used. You can manipulate them interactively and the function will be called. However, the widget instance returned by `interactive` also give you access to the current keyword arguments and return value of the underlying Python function.

Here are the current keyword arguments. If you rerun this cell after manipulating the sliders, the values will have changed.

In [36]:
w_sum.kwargs

{'a': 10, 'b': 20}

Here is the current return value of the function.

In [37]:
w_sum.result

30

### Using `Interactive` with `get_minutes`

In [38]:
def get_minutes(years=0, weeks=0, days=0, hours=1):
    'Calculates number of minutes in so many years, + weeks, + days'
    
    minutes = 60 * (hours + (24 * (days + (7 * weeks + (52 * years)))))
    print("{} years, {} weeks, {} days plus {} hours equal ... {} minutes"
          .format(years,weeks, days, hours, minutes))
    return minutes

w_get_mins= interactive(get_minutes, years=(0,150,1), weeks=(0,51,1), days=(0,6,1), hours=(0,23,1))

w_get_mins

0 years, 0 weeks, 4 days plus 1 hours equal ... 5820 minutes


5820

#### Note: We did not need to call `display` to see the widgets

In [39]:
type(w_get_mins)

ipywidgets.widgets.widget_box.Box

In [40]:
w_get_mins.children

(<ipywidgets.widgets.widget_int.IntSlider at 0x31788cb0>,
 <ipywidgets.widgets.widget_int.IntSlider at 0x31788b10>)

In [41]:
w_get_mins.kwargs

{'days': 4, 'hours': 1, 'weeks': 0, 'years': 0}

In [42]:
w_get_mins.kwargs['weeks']

0

#### Note: We added a return statement to `get_minutes` to enable `w_get_mins.result`

In [43]:
w_get_mins.result

5820

### Disabling continuous updates

When interacting with long running functions, realtime feedback is a burden instead of being helpful. See the following example:

In [44]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if 
                str(x)==str(x)[::-1] and 
                str(x**2)==str(x**2)[::-1]))
    return

In [45]:
%%time
slow_function(1e6)

1000000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002]
CPU times: user 5.93 s, sys: 0 ns, total: 5.93 s
Wall time: 5.93 s


Notice that the output is updated even while dragging the mouse on the slider. This is not useful for long running functions due to lagging:

In [46]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5))

5300000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002, 1000001, 1001001, 1002001, 1010101, 1011101, 1012101, 1100011, 1101011, 1102011, 1110111, 1111111, 2000002, 2001002]


There are two ways to mitigate this. You can either only execute on demand, or restrict execution to mouse release events.

### __manual

The __manual kwarg of interact allows you to restrict execution so it is only done on demand. A button is added to the interact controls that allows you to trigger an execute event.

In [47]:
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5),__manual=True)

<function __main__.slow_function>

### continuous_update

If you are using slider widgets, you can set the continuous_update kwarg to False. continuous_update is a kwarg of slider widgets that restricts executions to mouse release events.

In [31]:
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5,continuous_update=False))

8100000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002, 1000001, 1001001, 1002001, 1010101, 1011101, 1012101, 1100011, 1101011, 1102011, 1110111, 1111111, 2000002, 2001002]


### Arguments that are dependent of each other

Arguments that are dependent of each other can be expressed manually using observe. See the following example, where one variable is used to describe the bounds of another. For more information, please see the widget events example notebook.

In [48]:
x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

x_widget.color = 'black'
x_widget.slider_color = 'navy'

def update_x_range(*args):
    if y_widget.value > 8.0:
        x_widget.max = 16.0
        x_widget.color = 'red'
        x_widget.slider_color = 'red'
    else:
        x_widget.max = 10.0
        x_widget.color = 'green'
        x_widget.slider_color = 'green'

y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget)

0.0 5.0


<function __main__.printer>

#### Note: 

* I tried making `x_widget.slider_color`  dependent on `y_widget.value`, in the same way that `x_widget.color` is but without success


* [Table of Contents](#Table-of-Contents)

----